In [1]:
pip install azure-ai-textanalytics==5.2.0

In [13]:
import os
from azure.core.credentials import AzureKeyCredential
from azure.ai.textanalytics import TextAnalyticsClient

os.environ['VISION_KEY'] = "1iBEgn3st92HytC7Cc9RAnqYi0H2QBMCQCjYPlxgBAGDDLGrKwX1JQQJ99BEAC5RqLJXJ3w3AAAAACOG2ezo"
os.environ['VISION_ENDPOINT'] = "https://lab8aitextminingsentiments.openai.azure.com/"

subscription_key = os.environ["VISION_KEY"]
endpoint = os.environ["VISION_ENDPOINT"]

client = TextAnalyticsClient(endpoint=endpoint, credential=AzureKeyCredential(subscription_key))

In [14]:
import os
import csv

crtDir = os.getcwd()
fileName = os.path.join(crtDir, 'data', 'reviews_mixed.csv')
print(fileName)

data = []
with open(fileName) as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    headers = next(csv_reader)
    text_index = headers.index("Text")

    for row in csv_reader:
        data.append(row[text_index])

def chunks(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

batch_size = 10
for batch in chunks(data, batch_size):
    result = client.analyze_sentiment(batch, show_opinion_mining=True)
    docs = [doc for doc in result if not doc.is_error]

    for idx, doc in enumerate(docs):
        print(f"\nText: {batch[idx]}")
        print(f"Overall sentiment: {doc.sentiment}")


/content/data/reviews_mixed.csv

Text: The rooms are extremely small, practically only a bed.
Overall sentiment: negative

Text: Room safe did not work.
Overall sentiment: negative

Text: Mattress very comfortable.
Overall sentiment: positive

Text: Very uncomfortable, thin mattress, with plastic cover that rustles every time you move.
Overall sentiment: negative

Text: No bathroom in room
Overall sentiment: negative

Text: The bed was soooo comfy.
Overall sentiment: positive

Text: someone must have been smoking in the room next door.
Overall sentiment: neutral

Text: The bed is very comfortable.
Overall sentiment: positive

Text: Very spacious rooms, quiet and very comfortable.
Overall sentiment: positive

Text: For 3 people in a bedroom the sofa bed is a bit unconfortable.
Overall sentiment: negative

Text: Lights in the common room were too dim.
Overall sentiment: negative

Text: Air conditioning working fine.
Overall sentiment: positive

Text: So if you're the type that likes to l

In [15]:
message = [
    "By choosing a bike over a car, I’m reducing my environmental footprint. Cycling promotes eco-friendly transportation, and I’m proud to be part of that movement."
]

result = client.analyze_sentiment(message)[0]
print("Sentiment:", result.sentiment)
print("Confidence Scores:", result.confidence_scores)


Sentiment: positive
Confidence Scores: {'positive': 0.87, 'neutral': 0.13, 'negative': 0.0}


In [7]:
!pip install tensorflow


In [11]:
!pip install --upgrade numpy
!pip install --upgrade gensim

  Using cached numpy-2.2.5-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)
Using cached numpy-2.2.5-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (16.4 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gensim 4.3.3 requires numpy<2.0,>=1.18.5, but you have numpy 2.2.5 which is incompatible.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.2.5 which is incompatible.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 2.2.5 which is incompatible.
  Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.3 MB)
  A

In [9]:
!pip install gensim

In [13]:
!pip install --upgrade numpy
!pip install --upgrade --force-reinstall gensim

  Using cached numpy-2.2.5-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)
Using cached numpy-2.2.5-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (16.4 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gensim 4.3.3 requires numpy<2.0,>=1.18.5, but you have numpy 2.2.5 which is incompatible.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.2.5 which is incompatible.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 2.2.5 which is incompatible.
  Using cached gensim-4.3.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (8.1 kB)
  Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (

In [31]:
import os
import csv
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models import Word2Vec
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from scipy.sparse import hstack

# === Încărcare fișier CSV
crtDir = os.getcwd()
fileName = os.path.join(crtDir, 'data', 'reviews_mixed.csv')
texts, labels = [], []

with open(fileName, encoding='utf-8') as csv_file:
    reader = csv.reader(csv_file)
    headers = next(reader)
    text_idx = headers.index("Text")
    label_idx = headers.index("Sentiment")
    for row in reader:
        texts.append(row[text_idx])
        labels.append(row[label_idx])

# antrenare si test
trainInputs, testInputs, trainOutputs, testOutputs = train_test_split(
    texts, labels, test_size=0.2, random_state=42)

#TF-IDF
tfidf_vectorizer = TfidfVectorizer(max_features=100)
trainTFIDF = tfidf_vectorizer.fit_transform(trainInputs)
testTFIDF = tfidf_vectorizer.transform(testInputs)

#Word2Vec
tokenized_train = [t.lower().split() for t in trainInputs]
tokenized_test = [t.lower().split() for t in testInputs]
w2v_model = Word2Vec(sentences=tokenized_train, vector_size=50, window=5, min_count=1)

def vectorize_w2v(tokenized_texts, model):
    vecs = []
    for tokens in tokenized_texts:
        valid = [model.wv[t] for t in tokens if t in model.wv]
        vecs.append(np.mean(valid, axis=0) if valid else np.zeros(model.vector_size))
    return np.array(vecs)

trainW2V = vectorize_w2v(tokenized_train, w2v_model)
testW2V = vectorize_w2v(tokenized_test, w2v_model)

#alte caracteristici: nr cuvinte, lungime text, nr !
def custom_features(texts):
    return np.array([[len(t.split()), len(t), t.count('!')] for t in texts])

trainCustom = custom_features(trainInputs)
testCustom = custom_features(testInputs)

#Combinare caracteristici
trainCombined = hstack([trainTFIDF, trainW2V, trainCustom])
testCombined = hstack([testTFIDF, testW2V, testCustom])

#Transformare etichete
encoder = LabelEncoder()
trainLabels = encoder.fit_transform(trainOutputs)
testLabels = encoder.transform(testOutputs)

#ann
model = Sequential()
model.add(Dense(128, input_dim=trainCombined.shape[1], activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(64, activation='relu'))
model.add(Dense(len(encoder.classes_), activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(trainCombined.toarray(), trainLabels, epochs=10, batch_size=16, verbose=1)

#Evaluare
loss, acc = model.evaluate(testCombined.toarray(), testLabels)
print(f"Test accuracy: {acc*100:.2f}%")

#predictie pentru mesajul dat
msg = ["By choosing a bike over a car, I’m reducing my environmental footprint. Cycling promotes eco-friendly transportation, and I’m proud to be part of that movement."]
msg_tfidf = tfidf_vectorizer.transform(msg)
msg_w2v = vectorize_w2v([msg[0].lower().split()], w2v_model)
msg_custom = custom_features(msg)
msg_combined = hstack([msg_tfidf, msg_w2v, msg_custom])

prediction = model.predict(msg_combined.toarray()).argmax()
predicted_label = encoder.inverse_transform([prediction])[0]

print("Predicted Sentiment:", predicted_label)


Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.6382 - loss: 0.8534
Epoch 2/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6127 - loss: 1.1140 
Epoch 3/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6135 - loss: 0.9826 
Epoch 4/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6154 - loss: 0.8531 
Epoch 5/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6864 - loss: 0.7120 
Epoch 6/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6133 - loss: 0.7208 
Epoch 7/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6401 - loss: 0.6829 
Epoch 8/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6371 - loss: 0.7084 
Epoch 9/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5625 - loss: 0.7431 
Epoch 10/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6706 - loss: 0.7962 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.7103 - loss: 0.6617 
Test accuracy: 69.05%
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
Predicted Sentiment: positiv

In [14]:
import numpy as np
import csv
import os
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models import Word2Vec
from sklearn.preprocessing import LabelEncoder
from scipy.sparse import hstack


In [15]:
# Functie de activare ReLU
def relu(x):
    return np.maximum(0, x)

# Derivata functiei ReLU
def relu_derivative(x):
    return (x > 0).astype(float)

# Functie de activare Softmax
def softmax(x):
    exp_x = np.exp(x - np.max(x, axis=1, keepdims=True))  # stabilizare numerica
    return exp_x / np.sum(exp_x, axis=1, keepdims=True)

# Derivata Softmax pentru backpropagation
def softmax_derivative(output, y):
    return output - y


Cod propriu ann

In [16]:
class ANN:
    def __init__(self, input_dim, hidden_dim, output_dim):
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim

        self.W1 = np.random.randn(self.input_dim, self.hidden_dim) * 0.01
        self.b1 = np.zeros((1, self.hidden_dim))

        self.W2 = np.random.randn(self.hidden_dim, self.output_dim) * 0.01
        self.b2 = np.zeros((1, self.output_dim))

    def forward(self, X):
        # Propagare inainte (forward pass)
        self.Z1 = np.dot(X, self.W1) + self.b1
        self.A1 = relu(self.Z1)  # Strat ascuns
        self.Z2 = np.dot(self.A1, self.W2) + self.b2
        self.A2 = softmax(self.Z2)  # Strat de iesire (Softmax)
        return self.A2

    def backward(self, X, y, learning_rate=0.01):
        # Calcularea erorii si actualizarea greutatilor (backpropagation)

        m = X.shape[0]  # Dimensiunea lotului de antrenament

        # Calcularea gradientului pentru stratul de iesire
        dZ2 = softmax_derivative(self.A2, y)
        dW2 = np.dot(self.A1.T, dZ2) / m
        db2 = np.sum(dZ2, axis=0, keepdims=True) / m

        # Calcularea gradientului pentru stratul ascuns
        dA1 = np.dot(dZ2, self.W2.T)
        dZ1 = dA1 * relu_derivative(self.Z1)
        dW1 = np.dot(X.T, dZ1) / m
        db1 = np.sum(dZ1, axis=0, keepdims=True) / m

        # Actualizarea greutatilor
        self.W1 -= learning_rate * dW1
        self.b1 -= learning_rate * db1
        self.W2 -= learning_rate * dW2
        self.b2 -= learning_rate * db2


In [17]:
# Funcția de antrenare
def train(model, X_train, y_train, epochs=1000, learning_rate=0.01):
    for epoch in range(epochs):
        model.forward(X_train)
        model.backward(X_train, y_train, learning_rate)

        if epoch % 100 == 0:
            loss = -np.mean(np.log(model.A2[np.arange(y_train.shape[0]), np.argmax(y_train, axis=1)]))
            print(f"Epoch {epoch}, Loss: {loss:.4f}")

# Funcția de predicție
def predict(model, X):
    output = model.forward(X)
    return np.argmax(output, axis=1)


In [22]:
# Încărcarea datelor din fișier CSV
fileName = os.path.join(os.getcwd(), 'data', 'reviews_mixed.csv')
texts, labels = [], []

with open(fileName, encoding='utf-8') as csv_file:
    reader = csv.reader(csv_file)
    headers = next(reader)
    text_idx = headers.index("Text")
    label_idx = headers.index("Sentiment")
    for row in reader:
        texts.append(row[text_idx])
        labels.append(row[label_idx])

# Împărțirea datelor în seturi de antrenare și test
X_train, X_test, y_train, y_test = train_test_split(texts, labels, test_size=0.2, random_state=42)

# Preprocesarea textelor (TF-IDF și alte caracteristici personalizate)
tfidf_vectorizer = TfidfVectorizer(max_features=100)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train).toarray()
X_test_tfidf = tfidf_vectorizer.transform(X_test).toarray()

# Crearea unui encoder pentru etichete
encoder = LabelEncoder()
y_train_enc = encoder.fit_transform(y_train)
y_test_enc = encoder.transform(y_test)

# One-hot encoding pentru etichetele de ieșire
y_train_onehot = np.eye(len(encoder.classes_))[y_train_enc]
y_test_onehot = np.eye(len(encoder.classes_))[y_test_enc]

# Alte trăsături (număr de cuvinte, lungime text, nr. de semne de exclamare)
def custom_features(texts):
    return np.array([[len(t.split()), len(t), t.count('!')] for t in texts])

X_train_custom = custom_features(X_train)
X_test_custom = custom_features(X_test)

# Combinarea trăsăturilor
X_train_combined = np.hstack([X_train_tfidf, X_train_custom])
X_test_combined = np.hstack([X_test_tfidf, X_test_custom])


In [23]:
# Crearea unui model ANN
model = ANN(input_dim=X_train_combined.shape[1], hidden_dim=128, output_dim=len(encoder.classes_))

# Antrenarea modelului
train(model, X_train_combined, y_train_onehot, epochs=1000, learning_rate=0.01)

# Evaluarea modelului pe setul de test
y_pred = predict(model, X_test_combined)

# Calcularea acurateței
accuracy = np.mean(y_pred == np.argmax(y_test_onehot, axis=1))
print(f"Test accuracy: {accuracy*100:.2f}%")

# Predicție pentru un mesaj dat
msg = ["By choosing a bike over a car, I’m reducing my environmental footprint."]
msg_tfidf = tfidf_vectorizer.transform(msg).toarray()
msg_custom = custom_features(msg)
msg_combined = np.hstack([msg_tfidf, msg_custom])

pred = predict(model, msg_combined)
predicted_label = encoder.inverse_transform(pred)[0]
print(f"Predicted Sentiment: {predicted_label}")


Epoch 0, Loss: 0.6764
Epoch 100, Loss: 0.6353
Epoch 200, Loss: 0.6328
Epoch 300, Loss: 0.6309
Epoch 400, Loss: 0.6293
Epoch 500, Loss: 0.6280
Epoch 600, Loss: 0.6269
Epoch 700, Loss: 0.6259
Epoch 800, Loss: 0.6250
Epoch 900, Loss: 0.6240
Test accuracy: 71.43%
Predicted Sentiment: negative


In [26]:
import numpy as np
import csv
import os
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models import Word2Vec
from sklearn.preprocessing import LabelEncoder
from scipy.sparse import hstack
from sklearn.metrics import accuracy_score

# Function to compute softmax (normalized between 0 and 1)
def softmax(x):
    exp_x = np.exp(x - np.max(x, axis=1, keepdims=True))
    return exp_x / np.sum(exp_x, axis=1, keepdims=True)

# Cross-entropy loss calculation
def cross_entropy(y_true, y_pred):
    return -np.sum(y_true * np.log(y_pred + 1e-8)) / y_true.shape[0]

class MultiClassANN:
    def __init__(self, input_dim, hidden_dim, output_dim, learning_rate=0.01):
        self.learning_rate = learning_rate
        self.weights_input_hidden = np.random.randn(input_dim, hidden_dim)
        self.bias_hidden = np.zeros((1, hidden_dim))
        self.weights_hidden_output = np.random.randn(hidden_dim, output_dim)
        self.bias_output = np.zeros((1, output_dim))

    # Forward pass through the network
    def forward(self, X):
        self.z1 = np.dot(X, self.weights_input_hidden) + self.bias_hidden
        self.a1 = np.tanh(self.z1)  # Hidden layer activation
        self.z2 = np.dot(self.a1, self.weights_hidden_output) + self.bias_output
        self.a2 = softmax(self.z2)  # Output layer activation
        return self.a2

    # Backward pass for gradient calculation and weight updates
    def backward(self, X, y_true, y_pred):
        d_z2 = y_pred - y_true
        d_weights_hidden_output = np.dot(self.a1.T, d_z2)
        d_bias_output = np.sum(d_z2, axis=0, keepdims=True)

        d_a1 = np.dot(d_z2, self.weights_hidden_output.T)
        d_z1 = d_a1 * (1 - np.tanh(self.z1) ** 2)  # Derivative of tanh activation
        d_weights_input_hidden = np.dot(X.T, d_z1)
        d_bias_hidden = np.sum(d_z1, axis=0, keepdims=True)

        # Update weights and biases
        self.weights_hidden_output -= self.learning_rate * d_weights_hidden_output
        self.bias_output -= self.learning_rate * d_bias_output
        self.weights_input_hidden -= self.learning_rate * d_weights_input_hidden
        self.bias_hidden -= self.learning_rate * d_bias_hidden

    # Training function with specified epochs
    def train(self, X, y, epochs=1000):
        for epoch in range(epochs):
            y_pred = self.forward(X)
            loss = cross_entropy(y, y_pred)
            self.backward(X, y, y_pred)
            if epoch % 100 == 0:
                print(f"Epoch {epoch} - Loss: {loss:.4f}")

    # Prediction function
    def predict(self, X):
        probs = self.forward(X)
        return np.argmax(probs, axis=1)

# Preprocessing the message and predicting sentiment/emotion
def predict_emotion(message, model, word2vec_model, label_encoder):
    # Assuming preprocess_text and compute_features are defined elsewhere
    # and that word2vec_model is a trained Word2Vec model
    processed_message = preprocess_text(message)
    message_vector = compute_features(word2vec_model, [processed_message])
    prediction_index = model.predict(np.array(message_vector))[0]
    predicted_emotion = label_encoder.inverse_transform([prediction_index])[0]
    return predicted_emotion

# Load data
fileName = os.path.join(os.getcwd(), 'data', 'reviews_mixed.csv')
texts, labels = [], []

with open(fileName, encoding='utf-8') as csv_file:
    reader = csv.reader(csv_file)
    headers = next(reader)
    text_idx = headers.index("Text")
    label_idx = headers.index("Sentiment")
    for row in reader:
        texts.append(row[text_idx])
        labels.append(row[label_idx])

# Split data
X_train, X_test, y_train, y_test = train_test_split(texts, labels, test_size=0.2, random_state=42)

# Create Word2Vec model
tokenized_train = [t.lower().split() for t in X_train]
word2vec_model = Word2Vec(sentences=tokenized_train, vector_size=50, window=5, min_count=1)

# Create LabelEncoder
label_encoder = LabelEncoder()
train_labels_balanced = label_encoder.fit_transform(y_train)  # Assuming y_train is balanced

# Define compute_features (replace with your actual feature computation)
def compute_features(word2vec_model, texts):
    features = []
    for text in texts:
        tokens = text.lower().split()
        valid_tokens = [word2vec_model.wv[t] for t in tokens if t in word2vec_model.wv]
        text_feature = np.mean(valid_tokens, axis=0) if valid_tokens else np.zeros(word2vec_model.vector_size)
        features.append(text_feature)
    return features

# Compute features
train_features_balanced = compute_features(word2vec_model, X_train)
test_features = compute_features(word2vec_model, X_test)

# Define preprocess_text (replace with your actual text preprocessing)
def preprocess_text(text):
    return text.lower()  # Example: lowercasing

# Initialize and train the model
model = MultiClassANN(input_dim=word2vec_model.vector_size, hidden_dim=128, output_dim=len(label_encoder.classes_))

# Convert labels to one-hot encoding
one_hot_labels = np.eye(len(label_encoder.classes_))[train_labels_balanced]

# Train on balanced dataset
model.train(np.array(train_features_balanced), one_hot_labels, epochs=1000)

# Evaluate accuracy on test set
predictions = model.predict(np.array(test_features))
true_labels = label_encoder.transform(y_test)  # Transform y_test using the same encoder

# Calculate accuracy
accuracy = accuracy_score(true_labels, predictions)
print(f"Test accuracy: {accuracy:.2f}")

# Print predicted emotions in text
predicted_texts = label_encoder.inverse_transform(predictions)
print(f"Predictions (text): {predicted_texts}")

# Predict emotion for a new message
new_message = "By choosing a bike over a car, I’m reducing my environmental footprint. Cycling promotes eco-friendly transportation, and I’m proud to be part of that movement."
predicted_feeling = predict_emotion(new_message, model, word2vec_model, label_encoder)
print(f"Predicted emotion: {predicted_feeling}")

Epoch 0 - Loss: 0.6894
Epoch 100 - Loss: 5.3311
Epoch 200 - Loss: 3.1127
Epoch 300 - Loss: 5.9169
Epoch 400 - Loss: 1.8478
Epoch 500 - Loss: 8.3214
Epoch 600 - Loss: 5.9169
Epoch 700 - Loss: 5.8522
Epoch 800 - Loss: 5.9169
Epoch 900 - Loss: 5.9109
Test accuracy: 0.71
Predictions (text): ['negative' 'negative' 'negative' 'negative' 'negative' 'negative'
 'negative' 'negative' 'negative' 'negative' 'negative' 'negative'
 'negative' 'negative' 'negative' 'negative' 'negative' 'negative'
 'negative' 'negative' 'negative' 'negative' 'negative' 'negative'
 'negative' 'negative' 'negative' 'negative' 'negative' 'negative'
 'negative' 'negative' 'negative' 'negative' 'negative' 'negative'
 'negative' 'negative' 'negative' 'negative' 'negative' 'negative']
Predicted emotion: negative


In [1]:
from collections import Counter
print("Distribuția etichetelor în setul de antrenament:", Counter(y_train))
print("Distribuția etichetelor în setul de test:", Counter(y_test))


NameError: name 'y_train' is not defined